In [12]:
import os
import sys
import urllib.request
import json
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import numpy as np
import datetime
import pandas as pd

In [13]:
keywords = ["양궁",  "올림픽"]

results = {}

for keyword in keywords:

  result = []

  client_id = "mFVJrDtj4trdT2ermoVF"
  client_secret = "hbpIY84KD3"
  encText = urllib.parse.quote(keyword)

  for i in range(1, 1000, 100):
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + f'&display=100&start={i}' # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if (rescode==200):
        response_body = response.read()
        result += [response_body.decode('utf-8')]
        print(type(result))
    else:
        print("Error Code:" + rescode)
  results[keyword] = result

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [14]:
data['양궁']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:
import re

def processString(txt):
    specialChars = "!#$%^&*()"
    for specialChar in specialChars:
        txt = txt.replace(specialChar, '')
        txt = txt.replace(',', ' ')
        txt = txt.replace('.', ' ')
        txt = txt.replace('"', ' ')
        txt = txt.replace('ღ', ' ')
        txt = txt.replace('​', ' ')
        txt = txt.replace('  ', ' ')
        txt = txt.replace('   ', ' ')
        txt = txt.replace('    ', ' ')
        txt = txt.replace('     ', ' ')
    return txt.strip()

def processText(txt):
    specialChars = "!#$%^&*()"
    for specialChar in specialChars:
        txt = txt.replace('  ', ' ')
        txt = txt.replace('   ', ' ')
        txt = txt.replace('    ', ' ')
        txt = txt.replace('     ', ' ')
    return txt.strip()

# 본문에 키워드 언급 갯수
def keyword_search(txt):
    
    if(txt.find(keyword[0]) != -1):
        a = txt.find(keyword[0])
        b = 1
        while txt[a+1:].find(keyword) != -1:
            a = txt[a+1:].find(keyword) + a + 1
            b+=1
        return b

    

# 키워드가 본문의 처음 100단어 안에 포함되는가
def keyword_in_100(txt):
    a = txt.find("양궁")
    if(a<=100):
        return True
    else:
        return False

In [ ]:
import json
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import numpy as np
import datetime
import pandas as pd

data = np.array([])

for keyword in keywords:
  text1 = []
  text_amount = []
  keyword_mentioned = []
  items = []
  
  results_ = results[keyword]
  print(type(results_))
  for result in results_:
    print(type(result))
    items += json.loads(result)['items']
  #items = 블로그 배열 (items[0] = 첫번째 블로그)

  print(f'블로그 개수 : {len(items)}')

  

  for label, i in enumerate(range(0,len(items))):
    text2 = ""
    #print(i)
    response1 = requests.get(items[i]['link'])
    soup = BeautifulSoup(response1.text)
    idx = str(soup.find_all('iframe').pop()).find('src')
    source = str(soup.find_all('iframe').pop())[idx+5:-11]
    source = source.replace('amp;', '')
    if source.find('PostView.naver') == -1:
      continue
    url = 'http://blog.naver.com' + source
    response1 = requests.get(url)
    soup = BeautifulSoup(response1.text)
    # 예외처리
    # print(url)
    temp = soup.find_all("div", "se-main-container") + soup.find_all("div", id="postViewArea") + soup.find_all("div", id = "postListBody")
    temp1 = BeautifulSoup(str(temp.pop()))
    text = temp1.text.replace('\n', ' ')
    text1.append([processString(text)])
    title = items[i]['title']

    # 포스트 날짜
    postdate = datetime.datetime.strptime(items[i]['postdate'], '%Y%m%d')
    pastday = (datetime.datetime.now() - postdate).days

    #댓글 갯수
    comtemp = str(soup.find_all('em', id='floating_bottom_commentCount'))
    #print(comtemp)
    commentNum = BeautifulSoup(comtemp).text.replace(']', '').replace('[', '').strip()
    if commentNum == '' :
      commentNum =0
    else :
      commentNum = int(commentNum)

    # 키워드 언급 개수
    #keyword_mentioned.append([keyword_search(text)])
      
    # 본문 분량
    text2 += processText(text)
    text_amount.append(len(text2))
    
    #print("본문 분량:", text_amount[i], "자")
    #print("키워드 언급 횟수:", keyword_mentioned[i], "번")
    #print("댓글 개수:", commentNum,"개") 
    #print("본문의 처음 100단어안에 포함되는가:", keyword_in_100(text))

    if type(keyword) != list:
      keyword_list = keyword.split()
    keyword_title = []
    
    for k in keyword_list:
      keyword_title += [k in title]
    #print(f"제목에 키워드 포함 여부 : {keyword_list} : {keyword_title}")
    
    link_num = len(temp1.find_all('div', class_="se-module se-module-oglink"))
    #print(f'링크 개수 : {link_num}')
    
    player_num= len(temp1.find_all('div', class_= 'se-component se-video se-l-default') + temp1.find_all('a', class_="se-link"))
    # print(f'동영상 개수 : {player_num}')

    img_num= len(temp1.find_all('img'))
    # print(f'이미지 개수 : {img_num}')
    
    data = np.append(data, np.array([int(len(text2)), int(keyword_search(text)), int(commentNum), int(keyword_in_100(text)), int(link_num), int(player_num), int(img_num), int(pastday), int(label)]))



columns = ['본문길이', '키워드 언급개수', '댓글개수', '처음 100단어', '링크개수', '동영상개수', '이미지 개수', '지난 날짜', 'label']

data = data.reshape(-1, len(columns))

df = pd.DataFrame(data, columns = columns)
df.to_csv('data.csv')
df.head()

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd  = SGDRegressor()

In [ ]:
sgd.fit(df, label)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

class myModel(Model):
  def __init__(self):
    super(myModel, self).__init__()
    self.dense1 = Dense(100, activation = 'relu')
    self.dense2 = Dense(50, activation = 'relu')
    self.dense3 = Dense(30, activation = 'relu')
    self.dense4 = Dense(1, activation = 'linear')

  def call(self, x):
    return self.dense4(self.dense3(self.dense2(self.dense1(x))))

In [ ]:
model = myModel()

In [ ]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')

In [ ]:
@tf.function
def train_step(data, labels):
  with tf.GradientTape() as tape:
    predictions = model(data)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)

In [ ]:
@tf.function
def test_step(data, labels):
  predictions = model(data)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)

In [ ]:
df.to_numpy()

In [ ]:
print(len(df.to_numpy()), len(label))

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train_loss.result().numpy()

In [ ]:
label

In [ ]:
X_train, y_train

In [ ]:
EPOCHS = 1000
train_losses =[]
test_losses = []

In [ ]:
for epoch in range(EPOCHS):
  for data, labels in train_ds:
    train_step(data, labels)
  for test_x, test_labels in test_ds:
    test_step(test_x, test_labels)
  template = '에포크: {}, train_loss: {}, test_loss: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        test_loss.result()))
  train_losses += [train_loss.result().numpy()]
  test_losses += [test_loss.result().numpy()]

  if epoch % 100 == 0:
    plt.scatter(y_train, model(X_train))
    plt.show()


In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)
plt.legend(['train_loss', 'test_loss'])
plt.show()

In [ ]:
plt.scatter(y_train, model(X_train))
plt.grid()

In [ ]:
plt.scatter(y_train, model(X_train))
plt.show()

In [ ]:
model.losses

In [ ]:
data[0:1]

In [ ]:
data = df.to_numpy()

In [ ]:
import matplotlib.pyplot as plt


plt.scatter(label,model(data[0:1000]))
plt.xlabel('실제 블로그 순위')
plt.ylabel('예측 블로그 순위')
plt.show()

In [ ]:
for i in range(100):
  print(int(sgd.predict([data[i]])))

In [ ]:
data[0]

In [ ]:
data = data.reshape(-1,8)

In [ ]:
import pandas as pd

columns = ['본문길이', '키워드 언급개수', '댓글개수', '처음 100단어', '링크개수', '동영상개수', '이미지 개수', '지난 날짜']


data = pd.DataFrame(data, columns = columns)

In [ ]:
data

In [ ]:
data.to_csv('data')

In [ ]:
for a in data[:, 1]:
  a.pop()

In [ ]:
url = "http://blog.naver.com/PostView.naver?blogId=mahapass&logNo=222445754512&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"

response1 = requests.get(url)
soup = BeautifulSoup(response1.text)
  # 예외처리
print(url)
temp = soup.find_all("div", "se-main-container") + soup.find_all("div", id="postViewArea") + soup.find_all("div", id = "postListBody")
temp1 = BeautifulSoup(str(temp.pop()))
text = temp1.text.replace('\n', ' ')

In [ ]:
text

In [ ]:
data.reshape(-1, 7)

In [ ]:
from PyKomoran import Komoran, DEFAULT_MODEL
import konlpy
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

#komoran = Komoran(DEFAULT_MODEL['LIGHT'])
token_list = []
for text in text1:
  token_list += [hannanum.nouns(str(text1))]

print(len(token_list))
print(token_list[0])

블로그 글 작성 날짜

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://blog.naver.com/ht_seoge/222405714709'

response = requests.get(url)
soup=BeautifulSoup(response.text)
idx = str(soup.find_all('iframe').pop()).find('src')
source = str(soup.find_all('iframe').pop())[idx+5:-11]

source = source.replace('amp;', '')
url = 'http://blog.naver.com' + source

response = requests.get(url)
soup = BeautifulSoup(response.text)
date = soup.find_all("p", class_="date fil5 pcol2 _postAddDate") + soup.find_all("span", class_="se_publishDate pcol2") #+ soup.find_all("span", class_="date pcol2")
date = BeautifulSoup(str(date.pop())).text


date

이미지 개수

In [ ]:
url = 'https://blog.naver.com/ht_seoge/222405714709'

response = requests.get(url)
soup=BeautifulSoup(response.text)
idx = str(soup.find_all('iframe').pop()).find('src')
source = str(soup.find_all('iframe').pop())[idx+5:-11]

source = source.replace('amp;', '')
url = 'http://blog.naver.com' + source

response = requests.get(url)
soup = BeautifulSoup(response.text)
temp = soup.find_all("div", ["se-main-container"])
len(BeautifulSoup(str(temp)).find_all("img"))

In [ ]:
!pip install konlpy

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'

In [ ]:
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent) 

print(sent)
print(kospacing_sent)